<a href="https://colab.research.google.com/github/teshi24/aiso/blob/main/03_local_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Local Search Algorithms

Here, we want to find the shortest path connecting several cities considering the aerial distances. This problem is known as the "Traveling Sales(man) Problem".

If we have 15 cities to connect, we have 15! different possibilities. This is already bigger than 10^12. You can easily see that the problem becomes complex very quickly. We will have problems to systematically explore the search space. Therefore, we will use local search algorithms to tackle this problem.

Local search algorithms start with a solution and try to improve the solution by considering the neighbouring states. The best neighbour will be chosen until no better can be found.


Implement your local search algorithm of choice (your version of the simulated annealing, hill-climing or genetic algorithm) to find the shortest path connecting a list of cities!

For the **Testat**, you need to find the shortest path between the following cities:

`path = ['Sursee', 'Sion', 'Altdorf', 'Landquart', 'Konolfingen', 'Thun', 'Twann', 'Sargans', 'Lausanne', 'Vevey', 'Locarno', 'Hinwil', 'Bern', 'Liestal', 'Lugano']`


For each algorithm, I wrote down some hints and implementation suggestions below. You don't need to implement all algorithms to solve the testat exercise. Try to tweak your algorithm such that a good (or even best) solution is found.


In [1]:
!git clone https://github.com/iaherzog/search.git

Cloning into 'search'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 21 (delta 5), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (21/21), 594.13 KiB | 2.86 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [2]:
import sys
sys.path.append('/content/search')

## General hints

 You can use the following helper functions to plot visualize your path and to evaluate its cost.

In [3]:
import folium

def create_map(path, sbb):
    map = folium.Map(location=[46.8, 8.33],
                    zoom_start=8, tiles="Stamen TonerBackground")
    points = []
    first_city = path[0]
    previous_city = None
    for city in path:
        city_x = sbb.hubs[city].x
        city_y = sbb.hubs[city].y
        points.append([city_x, city_y])
        folium.Marker([city_x, city_y], popup=city).add_to(map)
        # if previous_city is not None:
            # Add PolyLine if previous city is not None
            # folium.PolyLine(locations=[[city_x, city_y], [previous_x, previous_y]], color='red').add_to(map)
        # previous_city = city  # Update previous_city for next iteration
        # previous_x, previous_y = city_x, city_y

    points.append([sbb.hubs[first_city].x, sbb.hubs[first_city].y])
    # folium.PolyLine(locations=[[sbb.hubs[first_city].x, sbb.hubs[first_city].y], [previous_x, previous_y]], color='red').add_to(map)
    folium.PolyLine(points, color='red').add_to(map)
    return map

def evaluate_path_original(path, distance_function):
    length = 0
    last_city = ""
    for city in path:
        if last_city == "":
            first_city = city;
        if last_city != "":
            length += distance_function(last_city, city)
        last_city = city;
    length += distance_function(first_city, last_city)
    return length


Let's import the data from sbb and viualize our initial path.

In [6]:
from sbb import SBB

sbb = SBB()
sbb.import_data('/content/search/linie-mit-betriebspunkten.json')
distance_function = sbb.get_distance_between

# initial path
path = ['Sursee', 'Sion', 'Altdorf', 'Landquart', 'Konolfingen', 'Thun', 'Twann', 'Sargans', 'Lausanne', 'Vevey', 'Locarno', 'Hinwil', 'Bern', 'Liestal', 'Lugano']
print("path cost = " + str(evaluate_path_original(path, distance_function)))

print(path)
m = create_map(path, sbb)
m

successfully imported 2787 hubs
successfully imported 401 train lines
path cost = 2009.1265150575216
['Sursee', 'Sion', 'Altdorf', 'Landquart', 'Konolfingen', 'Thun', 'Twann', 'Sargans', 'Lausanne', 'Vevey', 'Locarno', 'Hinwil', 'Bern', 'Liestal', 'Lugano']


This is defenitly not the best way how to connect the cities. Find the optimal solution with **one** of the follwoing algorithms.

## Genetic Algorithm

Genetic algorithms (or GA) are inspired by natural evolution and are particularly useful in optimization and search problems with large state spaces.

Given a problem, algorithms in the domain make use of a *population* of solutions (also called *states*), where each solution/state represents a feasible solution. At each iteration (often called *generation*), the population gets updated using methods inspired by biology and evolution, like *crossover*, *mutation* and *natural selection*.

A genetic algorithm works in the following way:

1) Initialize random population.

2) Calculate population fitness.

3) Select individuals for mating.

4) Mate selected individuals to produce new population.

     * Random chance to mutate individuals.

5) Repeat from step 2) until an individual is fit enough or the maximum number of iterations was reached.

Below, you can find some helper functions to implement your genetic algorithm.

First, create a dictionnary that maps a letter to a city name.

Our solution will be a path through all the cities. To simplify, we will encode each city with a letter from the alphabet. So your first initial path through the cities will have the code "ABCDEFGHIJK..". We can easily convert a letter to a city by `letter2city('A')` or `city2letter('Rotkreuz')`.

In [7]:
import string

number_of_cities = len(path)
letter2city = dict()
city2letter = dict()

for i in range(number_of_cities):
    letter2city[string.ascii_uppercase[i]] = list(path)[i]
    city2letter[list(path)[i]] = string.ascii_uppercase[i]

def path2string_original(path):
    s = ""
    for city in path:
        s+=city2letter[city]
    return list(s)

def path2cities_original(path):
    s = list()
    for letter in path:
        s.append(letter2city[letter])
    return s


def show_path_original(path):
  path_code = path2string_original(path)
  print("the path looks like this : ")
  print(path)
  print("the path has the following code : ")
  print(path_code)

show_path_original(path)

the path looks like this : 
['Sursee', 'Sion', 'Altdorf', 'Landquart', 'Konolfingen', 'Thun', 'Twann', 'Sargans', 'Lausanne', 'Vevey', 'Locarno', 'Hinwil', 'Bern', 'Liestal', 'Lugano']
the path has the following code : 
['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O']


Let's inizialize a random population:

In [9]:
import random

def init_population(pop_number, cities):
    population = []
    for _ in range(pop_number):
        individual = list(cities)
        random.shuffle(individual)
        population.append(individual)
    return population

cities = ['Sursee', 'Sion', 'Altdorf', 'Landquart', 'Konolfingen', 'Thun', 'Twann', 'Sargans', 'Lausanne', 'Vevey', 'Locarno', 'Hinwil', 'Bern', 'Liestal', 'Lugano']

for code_path in init_population(6, path2string_original(cities)):
    print(code_path)


['K', 'H', 'E', 'N', 'D', 'L', 'F', 'O', 'M', 'J', 'B', 'I', 'G', 'C', 'A']
['G', 'A', 'O', 'M', 'N', 'J', 'E', 'L', 'H', 'C', 'K', 'B', 'D', 'F', 'I']
['K', 'I', 'L', 'O', 'C', 'B', 'G', 'E', 'H', 'M', 'J', 'D', 'N', 'A', 'F']
['I', 'G', 'N', 'M', 'C', 'O', 'K', 'L', 'F', 'D', 'A', 'B', 'H', 'J', 'E']
['A', 'I', 'M', 'F', 'K', 'C', 'J', 'L', 'O', 'N', 'H', 'E', 'D', 'G', 'B']
['K', 'C', 'M', 'B', 'J', 'I', 'A', 'H', 'O', 'D', 'E', 'L', 'G', 'F', 'N']


We can calculate the fitness of a path using the `evaluate_path` function. Note that shorter paths are considered fitter.

In [11]:
def fitness(sample_route, distance_cache, letter2city):
    total_distance = evaluate_path_original(path2cities_original(sample_route, letter2city), distance_cache)
    return 1 / total_distance


Create a function to select two individuals for mating. Fitter individuals are more likely to be selected for reproduction than less fit individuals. Therefore, we have to calculate the weights of each indiviudal that corresponds to the likelyhood of being chosen for reproduction.

Now that we can select two individuals, we make them reproduce using crossover and mutation. We need to consider that we want to visit every city exactly once. For example, for the crossover, you can take a random lenght of individual 1 and fill up the remaining cities based on the order of the unvisited cities in individual 2.

In [13]:
def generate_child(parent1, parent2, split, missing_items):
    child = parent1[:split]
    child_set = set(child)
    for item in parent2:
        if item not in child_set:
            child.append(item)
            missing_items -= 1
            if missing_items <= 0:
                break
    return child


def crossover(x, y):
    len_x = len(x)
    split1, split2 = sorted(random.sample(range(len_x), 2))
    split = int((split1 + split2) / 2)
    missing_items = len_x - split

    child1 = generate_child(x, y, split, missing_items)
    child2 = generate_child(y, x, split, missing_items)

    return child1, child2

def mutate(route, mutation_rate):
    if random.random() < mutation_rate:
        idx1, idx2 = random.sample(range(len(route)), 2)
        route[idx1], route[idx2] = route[idx2], route[idx1]
    return route

path_code = path2string_original(cities)

# test your code
x = path_code
y = random.sample(path_code, len(path_code))
xy1, xy2 = crossover(x,y)
print(x)
print(y)
print(xy1)
mutate(xy1, 0.5)
print(xy1)
print(xy2)
mutate(xy2, 0.5)
print(xy2)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O']
['I', 'B', 'O', 'C', 'G', 'A', 'H', 'F', 'N', 'K', 'L', 'D', 'M', 'J', 'E']
['A', 'B', 'C', 'D', 'E', 'I', 'O', 'G', 'H', 'F', 'N', 'K', 'L', 'M', 'J']
['A', 'B', 'C', 'D', 'E', 'I', 'O', 'G', 'H', 'F', 'N', 'K', 'L', 'M', 'J']
['I', 'B', 'O', 'C', 'G', 'A', 'D', 'E', 'F', 'H', 'J', 'K', 'L', 'M', 'N']
['I', 'B', 'O', 'C', 'G', 'A', 'D', 'E', 'F', 'H', 'J', 'K', 'L', 'M', 'N']


We have now all the ingredients to create our genetic algorithm:

In [20]:
import heapq
import random
import concurrent.futures


def path2string(path, city2letter):
    s = ""
    for city in path:
        s += city2letter[city]
    return list(s)


def path2cities(path, letter2city):
    s = list()
    for letter in path:
        s.append(letter2city[letter])
    return s


def show_path(path, city2letter):
    path_code = path2string(path, city2letter)
    print("the path looks like this : ")
    print(path)
    print("the path has the following code : ")
    print(path_code)


class DistanceCache:
    def __init__(self, distance_function):
        self.distance_function = distance_function
        self.cache = {}

    def get_distance(self, city1, city2):
        key = tuple(sorted((city1, city2)))
        if key not in self.cache:
            self.cache[key] = self.distance_function(city1, city2)
        return self.cache[key]


def evaluate_path(path, distance_cache):
    length = 0
    last_city = path[-1]
    for city in path:
        if last_city != city:
            length += distance_cache.get_distance(last_city, city)
        last_city = city
    length += distance_cache.get_distance(path[0], last_city)
    return length


def init_population(pop_number, cities):
    population = []
    for _ in range(pop_number):
        individual = list(cities)
        random.shuffle(individual)
        population.append(individual)
    return population


def fitness(sample_route, distance_cache, letter2city):
    total_distance = evaluate_path(path2cities(sample_route, letter2city), distance_cache)
    return 1 / total_distance


def evaluate_fitness_wrapper(args):
    index, sample_route, distance_cache, letter2city = args
    return index, fitness(sample_route, distance_cache, letter2city)


def parallel_fitness_evaluation(population, distance_cache, best_fitness, best_route, letter2city, old_elites,
                                elite_count):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        population.extend([elite[1] for elite in old_elites])
        args = [(i, route, distance_cache, letter2city) for i, route in enumerate(population)]
        fitness_results = executor.map(evaluate_fitness_wrapper, args)
        fitness_values = [None] * len(population)
        elites = []
        heapq.heappush(elites, (best_fitness, best_route))
        least_fits = []
        least_fits_count = len(old_elites)
        for index, fitness_value in fitness_results:
            fitness_values[index] = fitness_value
            if len(elites) < elite_count or fitness_value > elites[0][0]:
                if len(elites) == elite_count:
                    heapq.heapreplace(elites, (fitness_value, population[index]))
                else:
                    heapq.heappush(elites, (fitness_value, population[index]))
            if len(least_fits) < least_fits_count or fitness_value < -least_fits[0][0]:
                if len(least_fits) == least_fits_count:
                    heapq.heapreplace(least_fits, (-fitness_value, population[index], index))
                else:
                    heapq.heappush(least_fits, (-fitness_value, population[index], index))

        least_fits.sort(key=lambda x: x[2], reverse=True)
        for _, _, index in least_fits:
            del population[index]
            del fitness_values[index]

        best_fitness, best_route = max(elites)

        return fitness_values, best_fitness, best_route, population, elites


def generate_child(parent1, parent2, split, missing_items):
    child = parent1[:split]
    child_set = set(child)
    for item in parent2:
        if item not in child_set:
            child.append(item)
            missing_items -= 1
            if missing_items <= 0:
                break
    return child


def crossover(x, y):
    len_x = len(x)
    split1, split2 = sorted(random.sample(range(len_x), 2))
    split = int((split1 + split2) / 2)
    missing_items = len_x - split

    child1 = generate_child(x, y, split, missing_items)
    child2 = generate_child(y, x, split, missing_items)

    return child1, child2


def mutate(route, mutation_rate):
    if random.random() < mutation_rate:
        idx1, idx2 = random.sample(range(len(route)), 2)
        route[idx1], route[idx2] = route[idx2], route[idx1]
    return route


def generate_crossover_children_wrapper(args):
    population, fitness_values = args
    parent1, parent2 = selection_parallel(population, fitness_values)
    return crossover(parent1, parent2)


def mutate_wrapper(args):
    route, mutation_rate = args
    return mutate(route, mutation_rate)


def parallel_genetic_operators(population, fitness_values, mate_number, mutation_rate):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_children = []
        for i in range(mate_number):
            args = (population, fitness_values)
            future_children.append(executor.submit(generate_crossover_children_wrapper, args))

        new_population = []
        for future in future_children:
            children = future.result()
            new_population.append(children[0])
            new_population.append(children[1])

        future_mutations = []
        for route in new_population:
            args = (route, mutation_rate)
            future_mutations.append(executor.submit(mutate_wrapper, args))

        for i, future in enumerate(future_mutations):
            new_population[i] = future.result()

        return new_population


def selection_parallel(population, fitness_values):
    # weight is higher when fitness is higher, therefore, route is probably choosen more often
    return random.choices(population, weights=fitness_values, k=2)


def genetic_algorithm_parallel(initial_path, pop_number, generations, mutation_rate, distance_cache,
                               letter2city, elitism_rate):
    mate_number = int(pop_number / 2)
    elite_number = int(elitism_rate * pop_number)
    best_route = initial_path
    best_fitness = fitness(initial_path, distance_cache, letter2city)
    elites = [(best_fitness, best_route, -1)]
    population = init_population(pop_number, initial_path)
    fitness_values, best_fitness, best_route, population, elites = parallel_fitness_evaluation(population,
                                                                                               distance_cache,
                                                                                               best_fitness, best_route,
                                                                                               letter2city, elites,
                                                                                               elite_number)
    for i in range(generations):
        if i % 50 == 0:
            print(f'iteration {i}, still processing... ')
        population = parallel_genetic_operators(population, fitness_values, mate_number, mutation_rate)
        fitness_values, best_fitness, best_route, population, elites = parallel_fitness_evaluation(population,
                                                                                                   distance_cache,
                                                                                                   best_fitness,
                                                                                                   best_route,
                                                                                                   letter2city, elites,
                                                                                                   elite_number)
    return best_route, best_fitness


if __name__ == '__main__':
    import string
    from sbb import SBB
    import time

    path = ['Sursee', 'Sion', 'Altdorf', 'Landquart', 'Konolfingen', 'Thun', 'Twann', 'Sargans', 'Lausanne', 'Vevey',
            'Locarno', 'Hinwil', 'Bern', 'Liestal', 'Lugano']
    number_of_cities = len(path)
    letter2city = dict()
    city2letter = dict()
    for i in range(number_of_cities):
        letter2city[string.ascii_uppercase[i]] = list(path)[i]
        city2letter[list(path)[i]] = string.ascii_uppercase[i]

    sbb = SBB()
    sbb.import_data('search/linie-mit-betriebspunkten.json')
    distance_function = sbb.get_distance_between
    distance_cache = DistanceCache(distance_function)

    print('start path')
    show_path(path, city2letter)
    print("path cost = " + str(evaluate_path(path, distance_cache)))

    start_time = time.time()
    path_code, best_fitness = genetic_algorithm_parallel(path2string(path, city2letter), pop_number=200,
                                                         generations=700,
                                                         mutation_rate=0.01, distance_cache=distance_cache,
                                                         letter2city=letter2city, elitism_rate=0.05)
    end_time = time.time()
    execution_time = end_time - start_time
    print("Execution time parallel:", execution_time, "seconds")
    print("Best route:", path_code)
    print("Best fitness:", best_fitness)

    path = path2cities(path_code, letter2city)

    print('--------------------------------')
    print('end path')
    show_path(path, city2letter)
    print("path cost = " + str(evaluate_path(path, distance_cache)))

    m = create_map(path, sbb)
    m

successfully imported 2787 hubs
successfully imported 401 train lines
start path
the path looks like this : 
['Sursee', 'Sion', 'Altdorf', 'Landquart', 'Konolfingen', 'Thun', 'Twann', 'Sargans', 'Lausanne', 'Vevey', 'Locarno', 'Hinwil', 'Bern', 'Liestal', 'Lugano']
the path has the following code : 
['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O']
path cost = 2153.3098883258394
iteration 0, still processing... 
iteration 50, still processing... 
iteration 100, still processing... 
iteration 150, still processing... 
iteration 200, still processing... 
iteration 250, still processing... 
iteration 300, still processing... 
iteration 350, still processing... 
iteration 400, still processing... 
iteration 450, still processing... 
iteration 500, still processing... 
iteration 550, still processing... 
iteration 600, still processing... 
iteration 650, still processing... 
Execution time parallel: 15.699589967727661 seconds
Best route: ['C', 'A', 'N', 'G', 'M', '

## Solution
_BTW, due to the parallelism, I had to pass the letter2city stuff through the algorithm, otherwise I had errors locally._

The final algorithm works as follows:

1. initialize values which are used multiple times, so that they do not need to be calculated to often
2. evaluates the fitness of the given route and initialize it as the elite
3. initializes the population and executes the fitness evaluation for the population (parallelized, more details later)
4. execute for all generations:
 1. create new generation based on population (parallelized)
 2. evaluate fitness (parallelized)
5. return best found route

The generations are built like this:
1. generate crossover between the parrents (parallelized)
  1. parents choosen randomly, but parents with higher fitness will be used more often
  2. parents split as shown in theory - used the avg. of 2 random numbers for split, so that the splits are for sure taking parts of both parents into consideration. Note: ensured that no duplicates are possible
2. mutations are simple city-swaps, to not create duplicates in this step (parallelized)

The fitness evaluation is a bit special:
1. it knows the elites from the previous generation and adds them to the population. This makes the algorithm which follows easier. As a tradeoff, the fitness for all elite members gets calculated multiple times. Since the algorithm is operating under 20 seconds, this was still sufficient for this excercise IMHO.
2. it caluclates the fitness values for all individuals (incl. old elite) (parallelized)
  1. the fitness is the value 1 / total_distance => this way, we get a bigger fitness value for smaller paths
3. when all fitness values got calculated, the algorithm continues sequentially and evaluates the provided values
  1. it assignes the fitness_value to the same index in fitness_values as the index in the popluation, so that the routes and their fitness do not get mixed up. (for that, the index was also given to the parallelized fitness calculator process)
  2. it saves as many elites as asked (relevant params: elitism_rate and pop_number). The elites are those routes with the best fits (= the shortest paths)
  3. it also saves the least fit items and removes them from the population at the end. This is not only required, since every generation the elites from the old generation are saved - this would increase our population size. It is also a nice benefit, since it removes those items from the generation, which show the largest path. This time, the same amount of items get removed like we added before (= count of old elites)
  4. it further provides us with the best route and its fitness; this is simply the maximum item from the elites list






In [21]:
# the best solution which was found with the algorithm:
path_code = ['K', 'B', 'J', 'I', 'G', 'M', 'F', 'E', 'N', 'A', 'C', 'L', 'H', 'D', 'O']
path = path2cities_original(path_code)

print('--------------------------------')
print('end path')
show_path_original(path)
print("path cost = " + str(evaluate_path_original(path, distance_function)))
m = create_map(path, sbb)
m

--------------------------------
end path
the path looks like this : 
['Locarno', 'Sion', 'Vevey', 'Lausanne', 'Twann', 'Bern', 'Thun', 'Konolfingen', 'Liestal', 'Sursee', 'Altdorf', 'Hinwil', 'Sargans', 'Landquart', 'Lugano']
the path has the following code : 
['K', 'B', 'J', 'I', 'G', 'M', 'F', 'E', 'N', 'A', 'C', 'L', 'H', 'D', 'O']
path cost = 805.6639692170133
